<a href="https://colab.research.google.com/github/zrruziev/deep_learning_practice/blob/main/PYTORCH/pytorch_official/001_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [1]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Download the Dataset

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### Prepare DataLoaders

In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [BS, C, H, W]: {X.shape}")
    print(f"Shape of y {y.shape}, {y.dtype}")
    break

Shape of X [BS, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]), torch.int64


### Create a Model

In [4]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available()else "cpu"

# Fashion_MNIST dataset has 10 classes
num_of_classes = 10

# Define model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, num_of_classes),
        )
    def forward(self, inputs):
        x = self.flatten(inputs)
        logits = self.linear_relu_stack(x)
        return logits

print(f"device: {device}\n")    
model = SimpleNN().to(device)
print(model)
        
        

device: cuda

SimpleNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Define loss and optimizer

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Train the model

In [6]:
epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n----------------------------")
    
    # Train Loop
    size = len(train_dataloader.dataset)
    # Set train mode
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        #Compute pred error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print some logs
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"\rloss: {loss:>7f} [{current:>5d}/{size:>5d}]", end="")
            
    # Validation Loop
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    # Set evaluation mode
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f" ==> Avg loss: {test_loss:>8f}  Accuracy: {correct:.0%} \n")

Epoch 1
----------------------------
loss: 2.168497 [57600/60000] ==> Avg loss: 2.158570  Accuracy: 46% 

Epoch 2
----------------------------
loss: 1.906155 [57600/60000] ==> Avg loss: 1.892253  Accuracy: 53% 

Epoch 3
----------------------------
loss: 1.519749 [57600/60000] ==> Avg loss: 1.522602  Accuracy: 60% 

Epoch 4
----------------------------
loss: 1.230730 [57600/60000] ==> Avg loss: 1.251867  Accuracy: 63% 

Epoch 5
----------------------------
loss: 1.063482 [57600/60000] ==> Avg loss: 1.084054  Accuracy: 65% 



### Save the model

In [7]:
torch.save(model.state_dict(), "model.pth")
print(f"Saved Model Path: {os.path.abspath('./model.pth')}")

Saved Model Path: /content/model.pth


### Load the model

In [8]:
model = SimpleNN()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

### Prediction

In [9]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

test_data_iter = iter(test_data)

n = 5
for i, (x, y) in enumerate(test_data_iter):
    if i == n:
        break
        
    with torch.inference_mode():  # torch.no_grad()
        pred = model(x)
        score = F.softmax(pred, dim=1).max()
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'\nActual    : {actual}\nPredicted : {predicted}-{score:.0%}')
    


Actual    : Ankle boot
Predicted : Ankle boot-43%

Actual    : Pullover
Predicted : Pullover-45%

Actual    : Trouser
Predicted : Trouser-78%

Actual    : Trouser
Predicted : Trouser-67%

Actual    : Shirt
Predicted : Shirt-23%
